In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
import functions as f 

%matplotlib inline




In [2]:
rent = pd.read_csv('data/csv/Campus_city/Campus_WMcitieslabels.csv', low_memory=False)

In [3]:
rent.shape

(2651885, 59)

## Find the dtype = object columns

In [4]:
rent.select_dtypes('object').columns

Index(['nebenkosten', 'heizkosten', 'letzte_modernisierung', 'nutzflaeche',
       'etage', 'anzahletagen', 'zimmeranzahl', 'nebenraeume', 'schlafzimmer',
       'badezimmer', 'ev_kennwert', 'click_customer', 'immobilientyp',
       'aufzug', 'balkon', 'einbaukueche', 'ev_wwenthalten', 'foerderung',
       'gaestewc', 'garten', 'heizkosten_in_wm_enthalten', 'keller',
       'parkplatz', 'rollstuhlgerecht', 'bauphase', 'ausstattung',
       'energieeffizienzklasse', 'energieausweistyp', 'haustier_erlaubt',
       'heizungsart', 'kategorie_Wohnung', 'objektzustand', 'lieferung',
       'bef1', 'bef2', 'bef5', 'bef4', 'bef3', 'anbieter', 'adat', 'edat'],
      dtype='object')

In [5]:
rent.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2651885 entries, 0 to 2651884
Data columns (total 59 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   obid                        2651885 non-null  int64  
 1   mietekalt                   2651885 non-null  float64
 2   nebenkosten                 2651885 non-null  object 
 3   heizkosten                  2651885 non-null  object 
 4   baujahr                     1682820 non-null  float64
 5   letzte_modernisierung       2651885 non-null  object 
 6   wohnflaeche                 2651885 non-null  float64
 7   nutzflaeche                 2651885 non-null  object 
 8   etage                       2651885 non-null  object 
 9   anzahletagen                2651885 non-null  object 
 10  zimmeranzahl                2651885 non-null  object 
 11  nebenraeume                 2651885 non-null  object 
 12  schlafzimmer                2651885 non-null  object 
 1

### Createnull_counts=copy of orginal dataframe for cleaning process

In [6]:
rent_cleaned = rent.copy()

### Map city names to ags_2019 column

In [7]:
ags_map = {11000000: 'Berlin', 6412000: 'Frankfurt', 2000000: 'Hamburg',
 5111000: 'Düsseldorf', 14612000: 'Dresden', 9162000: 'München',
 5315000: 'Köln', 5112000: 'Duisburg', 5913000: 'Dortmund',
 5113000: 'Essen', 4011000: 'Bremen', 3241001: 'Hannover', 
 8111000: 'Stuttgart', 14713000: 'Leipzig', 9564000: 'Nürnberg'
}

In [8]:
rent_cleaned.loc[:,'City'] = rent_cleaned.ags2019.map(ags_map)

In [9]:
rent_cleaned.ags2019.value_counts()

11000000    650962
14713000    195199
6412000     195165
2000000     195060
5111000     194902
14612000    194830
9162000     194800
5315000     194779
5112000      91067
5913000      90988
5113000      90912
4011000      90892
3241001      90890
8111000      90725
9564000      90714
Name: ags2019, dtype: int64

### Remove strings from object columns which should be np.nan

In [10]:
nan_values = ['Sonstiges Missing', 'keine Angabe', 'unplausibler Wert geloescht', 'Variable nicht mehr in Einagbemaske vorhanden' ]
rent_cleaned = rent_cleaned.replace(nan_values, np.nan)


### Convert object columns to numeric

In [11]:
float_columns = ['nebenkosten', 'heizkosten', 'letzte_modernisierung', 'nutzflaeche',
       'etage', 'anzahletagen', 'zimmeranzahl', 'nebenraeume', 'schlafzimmer',
       'badezimmer', 'ev_kennwert', 'click_customer']
rent_cleaned[float_columns] = rent_cleaned[float_columns].astype(float)


### Convert object columns to bool

In [12]:
rent_cleaned.select_dtypes('object').columns

Index(['immobilientyp', 'aufzug', 'balkon', 'einbaukueche', 'ev_wwenthalten',
       'foerderung', 'gaestewc', 'garten', 'heizkosten_in_wm_enthalten',
       'keller', 'parkplatz', 'rollstuhlgerecht', 'bauphase', 'ausstattung',
       'energieeffizienzklasse', 'energieausweistyp', 'haustier_erlaubt',
       'heizungsart', 'kategorie_Wohnung', 'objektzustand', 'lieferung',
       'bef1', 'bef2', 'bef5', 'bef4', 'bef3', 'anbieter', 'adat', 'edat',
       'City'],
      dtype='object')

In [13]:
bool_columns = ['aufzug', 'balkon', 'einbaukueche', 'ev_wwenthalten',
       'foerderung', 'gaestewc', 'garten', 'heizkosten_in_wm_enthalten',
       'keller', 'parkplatz', 'rollstuhlgerecht', 
       'haustier_erlaubt']

rent_cleaned[bool_columns] = rent_cleaned[bool_columns].apply(lambda x: x.map({'Ja':True, 'Nein':False}).astype(bool))

### Convert object columns to date time

In [14]:
rent_cleaned.adat = pd.to_datetime(rent_cleaned.adat, format='%Ym%m')
rent_cleaned.edat = pd.to_datetime(rent_cleaned.edat, format='%Ym%m')

In [15]:
rent_cleaned.select_dtypes('object').columns

Index(['immobilientyp', 'bauphase', 'ausstattung', 'energieeffizienzklasse',
       'energieausweistyp', 'heizungsart', 'kategorie_Wohnung',
       'objektzustand', 'lieferung', 'bef1', 'bef2', 'bef5', 'bef4', 'bef3',
       'anbieter', 'City'],
      dtype='object')

### Drop columns without information

In [16]:
drop_columns = ['bauphase', 'bef6', 'bef7', 'bef8', 'bef9', 'bef10']
rent_cleaned = rent_cleaned.drop(columns=drop_columns)


### Replace -9 in obejct columns to np.nan

In [17]:
firing_columns=['bef1', 'bef2', 'bef3', 'bef4', 'bef5']

In [18]:
rent_cleaned[firing_columns] = rent_cleaned[firing_columns].replace('-9', np.nan)

### Create dummy columns from Firing Type columns

In [19]:
# Define Set for unique firing types
firingTypes = set()

# loop over bef1 - bef5 columns
for col in firing_columns:
    # loop over firing types in bef
    for firing in rent_cleaned[col].value_counts().index:
        firingTypes.add(firing)
# Output of firing types
firingTypes


{'Bioenergie',
 'Elektro',
 'Erdgas leicht',
 'Erdgas schwer',
 'Erdwärme',
 'Fernwärme',
 'Flüssiggas',
 'Gas',
 'Heizdampf-Fernwärme',
 'Holz',
 'Holzschnitzel',
 'Kohle',
 'Koks',
 'Nahwärme',
 'Pellets',
 'Solarenergie',
 'Wasserenergie',
 'Windenergie',
 'Wärmezufuhr',
 'keine Information',
 'kombinierte Bio Energie',
 'kombinierte Fossilkraftstoffe',
 'kombinierte erneuerbare Energie',
 'kombinierte regenerative Energie',
 'thermische Umweltenergie',
 'Öl'}

In [20]:
# Create a bef df for each beff column with firing types dummies
# reindex bef df that each df has same columns
# fillna each dummy column with 0 and set astype int
bef = []
for col in firing_columns:
    bef.append(pd.get_dummies(rent_cleaned[col]).reindex(columns=firingTypes).fillna(0).astype(int))


In [21]:
# Sum all bef df and join this to rent_cleaned df
rent_cleaned = rent_cleaned.join(sum(bef))

In [22]:
# Drop no longer used bef columns
rent_cleaned = rent_cleaned.drop(columns=firing_columns)

In [23]:
rent_cleaned.select_dtypes('object')

,immobilientyp,ausstattung,energieeffizienzklasse,energieausweistyp,heizungsart,kategorie_Wohnung,objektzustand,lieferung,anbieter,City
0,Wohnung-Miete,Gehoben,NaN,NaN,Zentralheizung,Terassenwohnung,Neuwertig,Dez 2015,-9,Dresden
1,Wohnung-Miete,Gehoben,NaN,NaN,Zentralheizung,NaN,Neuwertig,Dez 2015,-9,Dresden
2,Wohnung-Miete,Gehoben,NaN,NaN,Zentralheizung,Dachgeschosswohnung,Neuwertig,Dez 2015,-9,Dresden
3,Wohnung-Miete,Gehoben,NaN,NaN,Zentralheizung,Dachgeschosswohnung,Neuwertig,Dez 2015,-9,Dresden
4,Wohnung-Miete,Gehoben,NaN,NaN,Zentralheizung,Dachgeschosswohnung,Neuwertig,Dez 2015,-9,Dresden
...,...,...,...,...,...,...,...,...,...,...
2651880,Wohnung-Miete,NaN,NaN,NaN,NaN,NaN,NaN,Feb 2020,Makler,Dortmund
2651881,Wohnung-Miete,NaN,NaN,NaN,NaN,Etagenwohnung,NaN,Feb 2020,Makler,Frankfurt
2651882,Wohnung-Miete,NaN,NaN,NaN,NaN,Etagenwohnung,NaN,Feb 2020,Makler,Frankfurt
2651883,Wohnung-Miete,Luxus,NaN,Endenergiebedarf,NaN,Etagenwohnung,Erstbezug,Jun 2020,Makler,Frankfurt


In [24]:
rent_cleaned.immobilientyp.value_counts()

Wohnung-Miete    2651885
Name: immobilientyp, dtype: int64

### Drop immonbilientyp column it is only one value and lieferung column it is only delivery data from Immoscout to rwi-essen


In [25]:
rent_cleaned = rent_cleaned.drop(columns=['immobilientyp', 'lieferung'])

### Replace -9 in anbieter column to np.nan

In [26]:
rent_cleaned.anbieter = rent_cleaned.anbieter.replace('-9', np.nan)


In [27]:
object_columns = rent_cleaned.select_dtypes('object').columns
object_columns

Index(['ausstattung', 'energieeffizienzklasse', 'energieausweistyp',
       'heizungsart', 'kategorie_Wohnung', 'objektzustand', 'anbieter',
       'City'],
      dtype='object')

### Drop columns nebenraeume because there are only 182 non-null values

In [28]:
rent_cleaned = rent_cleaned.drop(columns=['nebenraeume'])

### How to deal with the rest of missing values?

The columns `['etage', 'anzahletagen', 'zimmeranzahl', 'schlafzimmer', 'badezimmer', 'click_customer']` are set to -100 to seperate the nan values clearly from the rest of the values. 
For the other missing value columns the decission what is to do follows to the specific use.

In [67]:
rent_cleaned[['nebenkosten', 'heizkosten', 'baujahr', 'letzte_modernisierung', 
              'nutzflaeche', 'etage', 'anzahletagen', 'zimmeranzahl', 
              'schlafzimmer', 'badezimmer', 'ev_kennwert', 'click_customer']]

,nebenkosten,heizkosten,baujahr,letzte_modernisierung,nutzflaeche,etage,anzahletagen,zimmeranzahl,schlafzimmer,badezimmer,ev_kennwert,click_customer
0,140.0,NaN,1994.0,NaN,NaN,2.0,4.0,2.0,1.0,1.0,NaN,0.0
1,140.0,NaN,1994.0,NaN,NaN,2.0,4.0,2.0,1.0,1.0,NaN,0.0
2,140.0,NaN,1994.0,NaN,NaN,2.0,4.0,2.0,1.0,1.0,NaN,0.0
3,140.0,NaN,1994.0,NaN,NaN,2.0,4.0,2.0,1.0,1.0,NaN,0.0
4,140.0,NaN,1994.0,NaN,NaN,2.0,4.0,2.0,1.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2651880,90.0,NaN,1956.0,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN
2651881,330.0,NaN,2019.0,NaN,NaN,NaN,3.0,3.0,2.0,1.0,NaN,NaN
2651882,320.0,NaN,2019.0,NaN,NaN,1.0,3.0,3.0,2.0,1.0,NaN,NaN
2651883,330.0,NaN,2019.0,NaN,NaN,1.0,3.0,3.0,2.0,1.0,27.0,NaN


In [73]:
fillna_columns = ['etage', 'anzahletagen', 'zimmeranzahl', 'schlafzimmer', 
                  'badezimmer', 'ev_kennwert', 'click_customer']
rent_cleaned[fillna_columns] = rent_cleaned[fillna_columns].fillna(-100)

In [29]:
missing_value_columns = ['nebenkosten', 'heizkosten', 'baujahr', 
                         'letzte_modernisierung', 'nutzflaeche', 
                         'etage', 'anzahletagen', 'zimmeranzahl', 
                         'schlafzimmer', 'badezimmer', 'ev_kennwert', 
                         'click_customer']
rent_cleaned[missing_value_columns].describe()

,nebenkosten,heizkosten,baujahr,letzte_modernisierung,nutzflaeche,etage,anzahletagen,zimmeranzahl,schlafzimmer,badezimmer,ev_kennwert,click_customer
count,2.490467e+06,824489.000000,1.682820e+06,556602.000000,630468.000000,2.167254e+06,1.605995e+06,2.651445e+06,1.372067e+06,1.756338e+06,800264.000000,1.446675e+06
mean,1.454803e+02,72.841451,1.961778e+03,2010.758740,53.777122,2.363223e+00,4.144950e+00,2.483287e+00,1.535922e+00,1.085546e+00,122.571003,2.725250e+00
std,7.291253e+01,38.893883,3.739589e+01,6.197949,37.678995,1.948279e+00,2.385651e+00,8.917304e-01,7.218355e-01,2.900060e-01,52.630934,1.193625e+01
min,0.000000e+00,0.000000,1.500000e+03,1814.000000,0.000000,-1.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,9.500000e+01,50.000000,1.932000e+03,2009.000000,15.000000,1.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,85.099998,0.000000e+00
50%,1.300000e+02,68.000000,1.966000e+03,2012.000000,56.000000,2.000000e+00,4.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,118.000000,0.000000e+00
75%,1.800000e+02,90.000000,1.993000e+03,2015.000000,76.070000,3.000000e+00,5.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,152.399994,0.000000e+00
max,1.500000e+03,1500.000000,2.020000e+03,2018.000000,1000.000000,4.500000e+01,4.500000e+01,1.000000e+01,8.000000e+00,5.000000e+00,600.000000,2.645000e+03


#### Create year column

In [30]:
rent_cleaned['year'] = rent_cleaned.edat.dt.year



In [31]:
rent_cleaned.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2651885 entries, 0 to 2651884
Data columns (total 73 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   obid                              2651885 non-null  int64         
 1   mietekalt                         2651885 non-null  float64       
 2   nebenkosten                       2490467 non-null  float64       
 3   heizkosten                        824489 non-null   float64       
 4   baujahr                           1682820 non-null  float64       
 5   letzte_modernisierung             556602 non-null   float64       
 6   wohnflaeche                       2651885 non-null  float64       
 7   nutzflaeche                       630468 non-null   float64       
 8   etage                             2167254 non-null  float64       
 9   anzahletagen                      1605995 non-null  float64       
 10  zimmeranzahl      

### Pickle the dataframe


In [32]:

rent_cleaned.to_pickle('data/rent_cleaned.pkl')